In [3]:
from typing import List
from itertools import combinations
from random import sample, seed
import pandas as pd

# BCH(15,7,2) の生成多項式
G_BCH = [1,1,1,0,1,0,1,0,1]  # x^8 + x^7 + x^6 + x^4 + x^2 + 1

# CRC: g(x) = x^5 + x^3 + 1
G_CRC = [1, 0, 1, 0, 0, 1]

# mod2 除算
def poly_mod(dividend: List[int], divisor: List[int]) -> List[int]:
    dividend = dividend[:]
    while len(dividend) >= len(divisor):
        if dividend[0] == 1:
            for i in range(len(divisor)):
                dividend[i] ^= divisor[i]
        dividend.pop(0)
    return dividend

# BCH encode
def bch_encode(data: List[int]) -> List[int]:
    padded = data + [0] * (len(G_BCH) - len(data))
    remainder = poly_mod(padded, G_BCH)
    return data + remainder

# CRC encode
def crc_encode(data: List[int]) -> List[int]:
    padded = data + [0] * (len(G_CRC) - 1)
    remainder = poly_mod(padded, G_CRC)
    return data + remainder

# チェック関数
def bch_check(codeword: List[int]) -> bool:
    return all(b == 0 for b in poly_mod(codeword, G_BCH))

def crc_check(codeword: List[int]) -> bool:
    return all(b == 0 for b in poly_mod(codeword, G_CRC))

# エラー挿入
def inject_errors(bits: List[int], positions: List[int]) -> List[int]:
    corrupted = bits[:]
    for pos in positions:
        corrupted[pos] ^= 1
    return corrupted

# 実験本体
seed(42)  # 再現性のためのシード
results = []
data = [1,0,1,1,0,1,1]  # 7ビット固定
bch_code = bch_encode(data)
crc_code = crc_encode(data)

for error_bits in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    total_trials = 50
    bch_detect = 0
    crc_detect = 0
    for _ in range(total_trials):
        error_positions_bch = sample(range(len(bch_code)), error_bits)
        error_positions_crc = sample(range(len(crc_code)), error_bits)
        bch_corrupted = inject_errors(bch_code, error_positions_bch)
        crc_corrupted = inject_errors(crc_code, error_positions_crc)
        if not bch_check(bch_corrupted):
            bch_detect += 1
        if not crc_check(crc_corrupted):
            crc_detect += 1
    results.append({
        "誤りビット数": error_bits,
        "BCH誤り検出率": f"{bch_detect}/{total_trials}",
        "CRC誤り検出率": f"{crc_detect}/{total_trials}"
    })

# 表示
df = pd.DataFrame(results)
print(df.to_string(index=False))


 誤りビット数 BCH誤り検出率 CRC誤り検出率
      1    50/50    50/50
      2    47/50    50/50
      3    50/50    49/50
      4    50/50    49/50
      5    50/50    50/50
      6    50/50    48/50
      7    50/50    48/50
      8    49/50    49/50
      9    50/50    47/50
     10    50/50    48/50
